In [160]:
import pandas as pd
import re

In [161]:


# Ruta del archivo
Internet = r'C:\Users\claud\OneDrive\Escritorio\Henry_PI_DA\HENRY_PI2_Data_Analytics\Internet.xlsx'
# Cargar todas las hojas del archivo Excel
xls = pd.ExcelFile(Internet)
sheet_names = xls.sheet_names
sheet_names

['Accesos por velocidad',
 'Accesos Por Tecnología',
 'Penetración-poblacion',
 'Penetracion-hogares',
 'Dial-BAf',
 'Ingresos',
 'Accesos_tecnologia_localidad']

Cargo todas las hojas en un Diccionario

In [162]:
# Ruta del archivo
Internet = r'C:\Users\claud\OneDrive\Escritorio\Henry_PI_DA\HENRY_PI2_Data_Analytics\Internet.xlsx'

# Cargar todas las hojas en un diccionario de DataFrames
dfs = pd.read_excel(Internet, sheet_name=None)

# Mostrar las claves del diccionario (nombres de las hojas)
print(dfs.keys())


dict_keys(['Accesos por velocidad', 'Accesos Por Tecnología', 'Penetración-poblacion', 'Penetracion-hogares', 'Dial-BAf', 'Ingresos', 'Accesos_tecnologia_localidad'])


In [163]:
# Paso 1: Reemplazar celdas vacías por ceros
dfs_cleaned = {sheet_name: df.fillna(0) for sheet_name, df in dfs.items()}

In [164]:
# Listo de las hojas para hacer mi 1er visualización
hojas_a_ver = [
    'Accesos por velocidad',
    'Accesos Por Tecnología',
    'Penetración-poblacion',
    'Penetracion-hogares',
    'Dial-BAf',
    'Ingresos',
    'Accesos_tecnologia_localidad'
]

# Iterar a través de la lista de hojas y mostrar las primeras filas
for hoja in hojas_a_ver:
    if hoja in dfs_cleaned:
        print(f'\nHoja: {hoja}')
        print(dfs_cleaned[hoja].head())
    else:
        print(f'\nHoja: {hoja} no encontrada en el diccionario de DataFrames.')



Hoja: Accesos por velocidad
    Año  Trimestre        Provincia  HASTA 512 kbps  + 512 Kbps - 1 Mbps  \
0  2024          1     Buenos Aires         26002.0              22510.0   
1  2024          1  Capital Federal           516.0               5418.0   
2  2024          1        Catamarca            71.0                176.0   
3  2024          1            Chaco           236.0                416.0   
4  2024          1           Chubut           111.0               1083.0   

   + 1 Mbps - 6 Mbps  + 6 Mbps - 10 Mbps  + 10 Mbps - 20 Mbps  \
0           221185.0            233225.0             241778.0   
1            29753.0             51415.0              37369.0   
2             2872.0              3424.0               4204.0   
3            14156.0              9044.0               5533.0   
4            43828.0             31885.0              23024.0   

   + 20 Mbps - 30 Mbps  + 30 Mbps     OTROS      Total  
0             106418.0  4068292.0  125147.0  5044557.0  
1        

In [165]:
#Corregir errores en 'Accesos Por Tecnología', columna año y columna trimestre
if 'Accesos Por Tecnología' in dfs_cleaned:
    df_tec = dfs_cleaned['Accesos Por Tecnología']
    
    # Corregir la columna 'Año' (ejemplo: 2019* -> 2019)
    df_tec['Año'] = df_tec['Año'].replace({'2019*': '2019'}).astype(str)
    
    # Corregir la columna 'Trimestre' (ejemplo: 1*, 2*, 3* -> 1, 2, 3)
    df_tec['Trimestre'] = df_tec['Trimestre'].replace({'1*': '1', '2*': '2', '3*': '3'}).astype(str)
    
    # Actualizar el DataFrame en el diccionario
    dfs_cleaned['Accesos Por Tecnología'] = df_tec

In [166]:
if 'Accesos Por Tecnología' in dfs_cleaned:
    df_tec = dfs_cleaned['Accesos Por Tecnología']
    
    # Función para limpiar los asteriscos de los valores
    def clean_asterisk(value):
        if isinstance(value, str):
            return re.sub(r'\*', '', value)
        return value
    
    # Aplicar la función a la columna 'Año' y 'Trimestre'
    df_tec['Año'] = df_tec['Año'].apply(clean_asterisk)
    df_tec['Trimestre'] = df_tec['Trimestre'].apply(clean_asterisk)
    
    # Convertir las columnas a tipo numérico después de limpiar
    df_tec['Año'] = pd.to_numeric(df_tec['Año'], errors='coerce')
    df_tec['Trimestre'] = pd.to_numeric(df_tec['Trimestre'], errors='coerce')
    
    # Actualizar el DataFrame en el diccionario
    dfs_cleaned['Accesos Por Tecnología'] = df_tec

    # Verificar los cambios
    print("Valores únicos en la columna 'Año':")
    print(df_tec['Año'].unique())
    
    print("\nValores únicos en la columna 'Trimestre':")
    print(df_tec['Trimestre'].unique())


Valores únicos en la columna 'Año':
[2024 2023 2022 2021 2020 2019 2018 2017 2016 2015 2014]

Valores únicos en la columna 'Trimestre':
[1 4 3 2]


Se trabaja en la Uniformidad de columnas

In [167]:
# Normalizar nombres de columnas en todas las hojas
dfs_cleaned = {
    sheet_name: df.rename(columns=lambda x: x.strip().lower().replace(' ', '_'))
    for sheet_name, df in dfs_cleaned.items()
}

# Verificar los cambios en una hoja, por ejemplo, "Accesos por velocidad"
print(dfs_cleaned['Accesos por velocidad'].columns)

Index(['año', 'trimestre', 'provincia', 'hasta_512_kbps',
       '+_512_kbps_-_1_mbps', '+_1_mbps_-_6_mbps', '+_6_mbps_-_10_mbps',
       '+_10_mbps_-_20_mbps', '+_20_mbps_-_30_mbps', '+_30_mbps', 'otros',
       'total'],
      dtype='object')


In [168]:
# Reemplazar nulos por la media en columnas numéricas
for sheet_name, df in dfs_cleaned.items():
    df.fillna(df.mean(numeric_only=True), inplace=True)
    dfs_cleaned[sheet_name] = df


Correción en Tabla Ingresos

Veo en la columna Año que tiene mal el 20233 y se corrige, luego Ingresos (miles de pesos) se formatea para quitar notacion científica

In [169]:
# Asegurarse de que las columnas no tengan espacios o errores
df_ingresos = dfs_cleaned['Ingresos']

# Renombrar las columnas para estandarizar
df_ingresos.columns = df_ingresos.columns.str.strip().str.lower().str.replace(' ', '_')

# Mostrar nombres de columnas para verificar
print(df_ingresos.columns)

# Corregir el año incorrecto (2033 -> 2023)
df_ingresos['año'] = df_ingresos['año'].replace({2033: 2023})

# Formatear la columna 'Ingresos (miles de pesos)' para eliminar notación científica
df_ingresos['ingresos_(miles_de_pesos)'] = df_ingresos['ingresos_(miles_de_pesos)'].apply(lambda x: '{:,.2f}'.format(x))

# Verificar los resultados
print(df_ingresos)


Index(['año', 'trimestre', 'ingresos_(miles_de_pesos)', 'periodo'], dtype='object')
     año  trimestre ingresos_(miles_de_pesos)        periodo
0   2024          1            280,419,124.08   Ene-Mar 2024
1   2023          4            167,376,014.80   Oct-Dic 2023
2   2023          3            133,106,593.41  Jul-Sept 2023
3   2023          2            118,060,280.28   Jun-Mar 2023
4   2023          1            104,134,992.81   Ene-Mar 2023
5   2022          4             72,443,482.99   Oct-Dic 2022
6   2022          3             64,086,784.33  Jul-Sept 2022
7   2022          2             60,419,500.62   Abr-Jun 2022
8   2022          1             55,219,313.01   Ene-Mar 2022
9   2021          4             45,467,887.17   Oct-Dic 2021
10  2021          3             42,999,944.29  Jul-Sept 2021
11  2021          2             38,239,666.63   Abr-Jun 2021
12  2021          1             36,676,371.36   Ene-Mar 2021
13  2020          4             33,539,702.72   Oct-Dic 2020
1

Detección de Outliers

llevo los númeroa enteros, por preferencia

In [170]:
# Eliminar comas y convertir a formato numérico (float)
df_ingresos['ingresos_(miles_de_pesos)'] = df_ingresos['ingresos_(miles_de_pesos)'].replace({',': ''}, regex=True).astype(float)

# Convertir los valores a enteros
df_ingresos['ingresos_(miles_de_pesos)'] = df_ingresos['ingresos_(miles_de_pesos)'].astype(int)

# Verificar si la conversión fue exitosa
print(df_ingresos['ingresos_(miles_de_pesos)'].dtype)

# Ahora realizar el cálculo de IQR
Q1 = df_ingresos['ingresos_(miles_de_pesos)'].quantile(0.25)
Q3 = df_ingresos['ingresos_(miles_de_pesos)'].quantile(0.75)
IQR = Q3 - Q1

print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")


int64
Q1: 6912441.0, Q3: 42999944.0, IQR: 36087503.0


Hay una correcta distribución de los datos

In [171]:
# Crear la tabla de provincias con sus códigos INDEC
provincias_data = {
    'Código INDEC': [2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62, 66, 70, 74, 78, 82, 86, 90, 94],
    'Provincia': ['Capital Federal', 'Buenos Aires', 'Catamarca', 'Córdoba', 'Corrientes', 
                  'Chaco', 'Chubut', 'Entre Ríos', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 
                  'Mendoza', 'Misiones', 'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis', 
                  'Santa Cruz', 'Santa Fe', 'Santiago del Estero', 'Tierra del Fuego', 'Tucumán']
}

df_provincias = pd.DataFrame(provincias_data)

In [172]:
# Función para realizar el merge y agregar el Código INDEC a cada tabla con la columna Provincia
def agregar_codigo_indec(df):
    return df.merge(df_provincias, on='Provincia', how='left')

# Aplicar el merge a todas las tablas que tengan la columna 'Provincia'
for key in dfs.keys():
    if 'Provincia' in dfs[key].columns:
        dfs[key] = agregar_codigo_indec(dfs[key])

# Verificar que el Código INDEC se haya agregado correctamente
for key in dfs.keys():
    if 'Provincia' in dfs[key].columns:
        print(f"Tabla: {key}")
        print(dfs[key][['Provincia', 'Código INDEC']].head())

Tabla: Accesos por velocidad
         Provincia  Código INDEC
0     Buenos Aires           6.0
1  Capital Federal           2.0
2        Catamarca          10.0
3            Chaco          22.0
4           Chubut          26.0
Tabla: Accesos Por Tecnología
         Provincia  Código INDEC
0     Buenos Aires           6.0
1  Capital Federal           2.0
2        Catamarca          10.0
3            Chaco          22.0
4           Chubut          26.0
Tabla: Penetración-poblacion
         Provincia  Código INDEC
0     Buenos Aires           6.0
1  Capital Federal           2.0
2        Catamarca          10.0
3            Chaco          22.0
4           Chubut          26.0
Tabla: Penetracion-hogares
         Provincia  Código INDEC
0     Buenos Aires           6.0
1  Capital Federal           2.0
2        Catamarca          10.0
3            Chaco          22.0
4           Chubut          26.0
Tabla: Dial-BAf
         Provincia  Código INDEC
0     Buenos Aires           6.0
1  Capital 

Aplicar correcciones y Merger

Corrijo nombres, en este caso Buenos Aires, y El codigo para Capital Federal que por error no lo puse

In [173]:
# Función para realizar el merge y agregar el Código INDEC a cada tabla con la columna Provincia
def agregar_codigo_indec(df):
    df = corregir_nombres_provincias(df)  # Corregir los nombres antes del merge
    return df.merge(df_provincias, on='Provincia', how='left')

# Aplicar la corrección y el merge en las tablas con la columna 'Provincia'
for key in dfs.keys():
    if 'Provincia' in dfs[key].columns:
        dfs[key] = agregar_codigo_indec(dfs[key])

# Verificar si la columna 'Código INDEC' existe antes de intentar acceder a ella
for key in dfs.keys():
    if 'Provincia' in dfs[key].columns:
        if 'Código INDEC' in dfs[key].columns:
            print(f"Tabla: {key}")
            print(dfs[key][['Provincia', 'Código INDEC']].head())
        else:
            print(f"Advertencia: La columna 'Código INDEC' no está presente en la tabla '{key}'")

Advertencia: La columna 'Código INDEC' no está presente en la tabla 'Accesos por velocidad'
Advertencia: La columna 'Código INDEC' no está presente en la tabla 'Accesos Por Tecnología'
Advertencia: La columna 'Código INDEC' no está presente en la tabla 'Penetración-poblacion'
Advertencia: La columna 'Código INDEC' no está presente en la tabla 'Penetracion-hogares'
Advertencia: La columna 'Código INDEC' no está presente en la tabla 'Dial-BAf'
Advertencia: La columna 'Código INDEC' no está presente en la tabla 'Accesos_tecnologia_localidad'


In [174]:
# Mostrar las primeras filas de cada tabla en el diccionario dfs
for key in dfs.keys():
    print(f"Tabla: {key}")
    print(dfs[key].head())
    print("\n")

Tabla: Accesos por velocidad
    Año  Trimestre                        Provincia  HASTA 512 kbps  \
0  2024          1                     Buenos Aires         26002.0   
1  2024          1  Ciudad Autónoma de Buenos Aires           516.0   
2  2024          1                        Catamarca            71.0   
3  2024          1                            Chaco           236.0   
4  2024          1                           Chubut           111.0   

   + 512 Kbps - 1 Mbps  + 1 Mbps - 6 Mbps  + 6 Mbps - 10 Mbps  \
0              22510.0           221185.0            233225.0   
1               5418.0            29753.0             51415.0   
2                176.0             2872.0              3424.0   
3                416.0            14156.0              9044.0   
4               1083.0            43828.0             31885.0   

   + 10 Mbps - 20 Mbps  + 20 Mbps - 30 Mbps  + 30 Mbps     OTROS      Total  \
0             241778.0             106418.0  4068292.0  125147.0  5044557.

In [175]:
# Función para limpiar las columnas duplicadas y mantener solo 'Código INDEC'
def limpiar_columnas_indec(df):
    # Si hay columnas duplicadas, eliminarlas
    columns_to_drop = [col for col in df.columns if 'Código INDEC_' in col]
    df.drop(columns=columns_to_drop, inplace=True)
    return df

# Aplicar la limpieza a todas las tablas que tengan la columna 'Código INDEC'
for key in dfs.keys():
    if 'Código INDEC' in dfs[key].columns:
        dfs[key] = limpiar_columnas_indec(dfs[key])

# Verificar los resultados después de la limpieza
for key in dfs.keys():
    if 'Código INDEC' in dfs[key].columns:
        print(f"Tabla: {key}")
        print(dfs[key].head())


In [176]:
# Mostrar las primeras filas de cada tabla en el diccionario dfs
for key in dfs.keys():
    print(f"Tabla: {key}")
    print(dfs[key].head())
    print("\n")


Tabla: Accesos por velocidad
    Año  Trimestre                        Provincia  HASTA 512 kbps  \
0  2024          1                     Buenos Aires         26002.0   
1  2024          1  Ciudad Autónoma de Buenos Aires           516.0   
2  2024          1                        Catamarca            71.0   
3  2024          1                            Chaco           236.0   
4  2024          1                           Chubut           111.0   

   + 512 Kbps - 1 Mbps  + 1 Mbps - 6 Mbps  + 6 Mbps - 10 Mbps  \
0              22510.0           221185.0            233225.0   
1               5418.0            29753.0             51415.0   
2                176.0             2872.0              3424.0   
3                416.0            14156.0              9044.0   
4               1083.0            43828.0             31885.0   

   + 10 Mbps - 20 Mbps  + 20 Mbps - 30 Mbps  + 30 Mbps     OTROS      Total  \
0             241778.0             106418.0  4068292.0  125147.0  5044557.

In [177]:
# Ver si esta la columna en formato numerico
df_ingresos['ingresos_(miles_de_pesos)'] = pd.to_numeric(df_ingresos['ingresos_(miles_de_pesos)'], errors='coerce')

# Aplicar el formateo para eliminar la notación científica y mostrar los números con comas y dos decimales
df_ingresos['ingresos_(miles_de_pesos)'] = df_ingresos['ingresos_(miles_de_pesos)'].apply(lambda x: '{:,.2f}'.format(x))

# Verificar los resultados
print(df_ingresos)

     año  trimestre ingresos_(miles_de_pesos)        periodo
0   2024          1            280,419,124.00   Ene-Mar 2024
1   2023          4            167,376,014.00   Oct-Dic 2023
2   2023          3            133,106,593.00  Jul-Sept 2023
3   2023          2            118,060,280.00   Jun-Mar 2023
4   2023          1            104,134,992.00   Ene-Mar 2023
5   2022          4             72,443,482.00   Oct-Dic 2022
6   2022          3             64,086,784.00  Jul-Sept 2022
7   2022          2             60,419,500.00   Abr-Jun 2022
8   2022          1             55,219,313.00   Ene-Mar 2022
9   2021          4             45,467,887.00   Oct-Dic 2021
10  2021          3             42,999,944.00  Jul-Sept 2021
11  2021          2             38,239,666.00   Abr-Jun 2021
12  2021          1             36,676,371.00   Ene-Mar 2021
13  2020          4             33,539,702.00   Oct-Dic 2020
14  2020          3             31,997,444.00  Jul-Sept 2020
15  2020          2     

In [178]:
import os

# Crear la carpeta ETL si no existe
os.makedirs('ETL', exist_ok=True)

# Guardar cada tabla en un archivo CSV
for key in dfs.keys():
    dfs[key].to_csv(f'ETL/{key}.csv', index=False)

# Confirmación
"Tablas guardadas en la carpeta ETL como archivos CSV."

'Tablas guardadas en la carpeta ETL como archivos CSV.'